In [1]:
import math
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
# Constants
PI = math.pi

# User input
diameter = 2.5
a = 1.0
scale_down = 1.0
Lx = diameter / (a * scale_down)
Ly = 2.0 * Lx
resolution = 100
Lpml = 0.25
src_ypos = Ly - Lpml - 0.25
transmission_ypos = Lpml + 1.0
pol = 1
fmin_dft = 1.0
fmax_dft = 1.0
df_src = 1.0
f0_src = (fmin_dft + fmax_dft) / 2.0
theta_f0 = 0.0 * PI / 180.0
k_per = 2.0 * PI * f0_src * math.sin(theta_f0)
Nf_dft = 1
d = (diameter / (a * scale_down)) / 2.0
focal = d * math.sqrt(21.0) / 2.0
r_cur = focal / (2 * a * scale_down)
sin_NA = d / math.sqrt(d * d + focal * focal)
k_airy = 2.0 * PI / 1.0
airy_NA = 0.38317 * Lx / 2.0
lens_ypos = src_ypos - 0.5
x_center = Lx / 2.0
y_center = lens_ypos + math.sqrt(r_cur * r_cur - d * d)

# Functions
def eps_lens(pos):
    if pos[1] <= lens_ypos:
        if (pos[0] - x_center) ** 2 + (pos[1] - y_center) ** 2 <= r_cur ** 2:
            return 1.5 * 1.5
        else:
            return 1.0
    else:
        return 1.0

def eps_free(pos):
    return 1.0

# Power normalization
def power_normalization():
    power_norm = []

    v_free = mp.vol2d(Lx, Ly, resolution)
    s_free = mp.structure(v_free, eps_free, mp.pml(Lpml))
    f_free = mp.fields(s_free)

    p = mp.Plane(mp.XDIM, 0, Lx, src_ypos, resolution)
    k_nonp_f0 = math.sqrt(4.0 * PI * PI * f0_src * f0_src - k_per * k_per)
    k_inc = mp.Vec3(k_per, k_nonp_f0, 0)

    if pol == 0:
        mp.add_plane_wave(f_free, p, mp.Ez, k_inc, f0_src, df_src)
    else:
        mp.add_plane_wave(f_free, p, mp.Hz, k_inc, f0_src, df_src, math.cos(theta_f0))
        if theta_f0 > 0:
            mp.add_plane_wave(f_free, p, mp.Ey, k_inc, f0_src, df_src, math.sin(theta_f0))

    vol_bot = mp.volume(mp.vec(0, lens_ypos - 1.0), mp.vec(Lx, lens_ypos - 1.0))
    flux_bot = f_free.add_dft_flux_plane(vol_bot, fmin_dft, fmax_dft, Nf_dft)

    while f_free.time() < (f_free.last_source_time() + Ly):
        f_free.step()

    power_bot = flux_bot.flux()
    power_norm.append([abs(0.5 * power_bot[i]) for i in range(Nf_dft)])

    print("flux_free =", power_bot[0])

    with open("output/flux/flux_free.txt", "w") as fp:
        for val in power_norm[0]:
            fp.write(f"{val}\n")

    return power_norm

# Main optimization function
def main():

    # power_norm = power_normalization()
    power_mat = []

    v = mp.vol2d(Lx, Ly, resolution)
    s = mp.structure(v, eps_lens, mp.pml(Lpml))
    f = mp.fields(s)

    f.output_hdf5(mp.Dielectric, v.surroundings())
    
    sources = [
            mp.Source(
                mp.GaussianSource(fcen, fwidth=df),
                component=mp.Ez,
                center=mp.Vector3(0),
                size=mp.Vector3(Lx),
            ),
        mp.Source(
                mp.GaussianSource(fcen, fwidth=df),
                component=mp.Hz,
                center=mp.Vector3(0),
                size=mp.Vector3(Lx),
            )
        ]

#     p = mp.Plane(mp.XDIM, 0, Lx, src_ypos, resolution)
#     k_nonp_f0 = math.sqrt(4.0 * PI * PI * f0_src * f0_src - k_per * k_per)
#     k_inc = mp.Vec3(k_per, k_nonp_f0, 0)

#     if pol == 0:
#         mp.add_plane_wave(f, p, mp.Ez, k_inc, f0_src, df_src)
#     else:
#         mp.add_plane_wave(f, p, mp.Hz, k_inc, f0_src, df_src, math.cos(theta_f0))
#         if theta_f0 > 0:
#             mp.add_plane_wave(f, p, mp.Ey, k_inc, f0_src, df_src, math.sin(theta_f0))

    l_bot0 = mp.vec(Lx / 2 + airy_NA * 2.0, lens_ypos - focal)
    r_bot0 = mp.vec(Lx / 2 - airy_NA * 2.0, lens_ypos - focal)
    vol_bot0 = mp.volume(l_bot0, r_bot0)
    flux_bot0 = f.add_dft_flux_plane(vol_bot0, fmin_dft, fmax_dft, Nf_dft)

    l_bot1 = mp.vec(Lx / 2 + airy_NA, lens_ypos - focal)
    r_bot1 = mp.vec(Lx / 2 - airy_NA, lens_ypos - focal)
    vol_bot1 = mp.volume(l_bot1, r_bot1)
    flux_bot1 = f.add_dft_flux_plane(vol_bot1, fmin_dft, fmax_dft, Nf_dft)

    l_bot2 = mp.vec(0.0, lens_ypos - focal)
    r_bot2 = mp.vec(Lx, lens_ypos - focal)
    vol_bot2 = mp.volume(l_bot2, r_bot2)
    flux_bot2 = f.add_dft_flux_plane(vol_bot2, fmin_dft, fmax_dft, Nf_dft)

    l_bot3 = mp.vec(0.0, lens_ypos - (r_cur - math.sqrt(r_cur * r_cur - d * d)) - 1.0)
    r_bot3 = mp.vec(Lx, lens_ypos - (r_cur - math.sqrt(r_cur * r_cur - d * d)) - 1.0)
    vol_bot3 = mp.volume(l_bot3, r_bot3)
    flux_bot3 = f.add_dft_flux_plane(vol_bot3, fmin_dft, fmax_dft, Nf_dft)

    l_bot_x1 = mp.vec(0.1, 0.0)
    r_bot_x1 = mp.vec(0.1, Ly)
    vol_bot_x1 = mp.volume(l_bot_x1, r_bot_x1)
    flux_bot_x1 = f.add_dft_flux_plane(vol_bot_x1, fmin_dft, fmax_dft, Nf_dft)

    l_bot_x2 = mp.vec(Lx - 0.1, 0.0)
    r_bot_x2 = mp.vec(Lx - 0.1, Ly)
    vol_bot_x2 = mp.volume(l_bot_x2, r_bot_x2)
    flux_bot_x2 = f.add_dft_flux_plane(vol_bot_x2, fmin_dft, fmax_dft, Nf_dft)

    l_bot5 = mp.vec(Lpml, 0)
    r_bot5 = mp.vec(Lx - Lpml, lens_ypos - (r_cur - math.sqrt(r_cur * r_cur - d * d)))
    vol_bot5 = mp.volume(l_bot5, r_bot5)
    fields_bot = f.add_dft_fields([mp.Ex, mp.Ey, mp.Ez, mp.Hx, mp.Hy, mp.Hz], 6, vol_bot5, fmin_dft, fmax_dft,
                                   Nf_dft)
    flux_bot4 = f.add_dft_flux(mp.Y, vol_bot5, fmin_dft, fmax_dft, Nf_dft)

    while f.time() < (f.last_source_time() + Ly * 1.25):
        f.step()

    power_bot0 = flux_bot0.flux()
    power_bot = flux_bot1.flux()
    power_bot2 = flux_bot2.flux()
    power_bot3 = flux_bot3.flux()
    power_pml1 = flux_bot_x1.flux()
    power_pml2 = flux_bot_x2.flux()

    power_mat.append([abs(0.5 * power_bot0[i]) for i in range(Nf_dft)])

    print("NA0.4_airy_disk_flux =", power_bot0[0])
    print("Focal_point_flux =", power_bot[0])
    print("output_flux =", power_bot2[0])
    print("transmitted_flux =", power_bot3[0])
    print("Left_PML_flux =", power_pml1[0])
    print("Right_PML_flux =", power_pml2[0])

    with open("output/flux/flux_airy_disk.txt", "w") as fp6:
        for val in power_mat[0]:
            fp6.write(f"{val}\n")

    if pol == 0:
        f.output_dft(flux_bot4, "output/h5/dft-flux")
        f.output_dft(fields_bot, "output/h5/dft-fields")
    else:
        f.output_dft(flux_bot4, "output/h5/dft-flux")
        f.output_dft(fields_bot, "output/h5/dft-fields")

if __name__ == "__main__":
    main()



time for choose_chunkdivision = 0.000129938 s
time for set_epsilon = 0.199614 s
creating output file "./eps-000000000.h5"...


NameError: name 'fcen' is not defined

In [ ]:
#include <math.h>
#include <complex.h>
#include <time.h>
#include <ctime>
#include "meep.hpp"
#include "nlopt.hpp"
#include "../src/ale.hpp"
using namespace meep;
#define PI 3.14159265
// diameter = 5.0 -> focal point = 147.55(Matlab) -> Ly = 385 -> focal length =  231.45  = 0.9258 * focal
// diameter = 7.5 -> focal point = 98(Matlab) -> Ly = 260 -> focal length = 156 = 0.936 * focal
// diameter = 10 -> focal point = 75.4(Matlab) -> Ly = 197.5 -> focal length = 116.1 = 0.9288 * focal
// diameter = 12.5 -> focal point = 62.5(Matlab) -> Ly = 160 -> focal length = 91.5 = 0.915 * focal
// diameter = 15 -> focal point = 54.2(Matlab) -> Ly = 135 -> focal length = 74.8 = 0.8976 * focal
// diameter = 20 -> focal point = 37.3(Matlab) -> Ly = 103.75 -> focal length = 64.45 = 1.03 * focal
// diameter = 25 -> focal point = 16.95(Matlab) -> Ly = 85 -> focal length = 62.05 = 1.241 * focal
/////////////////////////2트////////////////////////////
// 50 -> 35.85(Matlab) -> 94 - 35.85 = focal length
// 25 ->
// 12.5 -> 6.15(Matalb) -> 19 - 6.15 = 12.85 = focal length(theory: 14.32)
///////////////////////// Start of the User input ///////////////////////////// 1a = 500nm, 50 x 60 volume(initial)
const double diameter = 2.5; // enter the diameter
const double a = 1.0; // 1a = 500nm(0.5), 1um(1.0)
const double scale_down = 1.0; // scale down factor
// geometry properties
const double Lx = diameter / (a * scale_down); // Lx -> simulation diameter -> 50 fix
const double Ly = 2.0 * Lx; // 10.0 + 25.0 * 1.25 / (a * scale_down); //
const double resolution = 100; // 1/resolution = a size of one pixel
const double Lpml = 0.25; // thickness of the PML
/////////////////////// About position information /////////////////////////
const double src_ypos = Ly - Lpml - 0.25; // position of the source -> y = Ly - 4.0
const double transmission_ypos = Lpml + 1.0; // position of the transmission -> calculate with NA index -> y = 4.0
////////////////////////////// source properties/////////////////////////////
const int pol = 1;  // polarization (0 = TE, 1 = TM)
const double fmin_dft = 1.0; // minimum f0_srcuency
const double fmax_dft = 1.0; // maximum f0_srcuency
const double df_src = 1.0; // bandwidth of the source
const double f0_src = (fmin_dft + fmax_dft) / 2.0;// center f0_srcuency
const double theta_f0 = 0.*PI/180.; // incidence angle degree to radian
const double k_per = 2. * PI * f0_src * sin(theta_f0);
const int Nf_dft = 1; // the number of DFT f0_srcuency
/////////////////////parameters of the Plano convex lens//////////////////////
const double d = (diameter / (a * scale_down)) / 2.0; // (diameter / scale_down) / 2 -> half of the diameter // fixed
const double focal = d * sqrt(21.0) / 2.0; // NA = 0.6, 100.0/3.0 = 33.33
const double r_cur = focal / (2 * a * scale_down); // focal / (2.0 * a * scale_down); // radius of curvature(5 fixed) -> f = R/(n-1) -> n = 1.5(fixed) -> f = 2R -> f = 10, R = 5
// const double sin_theta = d / sqrt(d * d + focal * focal); // (lens_ypos - 16.85) * (lens_ypos - 16.85) // NA = nsin(theta) -> n = 1(air) -> fix: when the diameter is 10
const double sin_NA = d / sqrt(d * d + focal * focal);
const double k_airy = 2.0 * PI / 1.0;
const double airy_NA = 0.38317 * Lx / 2.0; // 3.8317/(a * scale_down * k_airy * sin_NA);
// const double airy_disk = 3.8317/(k_airy * sin_theta); // make sure need I devide this parameter by scale factor? -> As the d is being smaller and smaller, airy_disk is bigger.
const double lens_ypos = src_ypos - 0.5;
const double x_center = Lx / 2.0; // x center position of the circle 
const double y_center = lens_ypos + sqrt(r_cur * r_cur - d * d); // y center position of the circle -> change with diameter

// f = 12um, diameter = 10um -> NA = nsin(theta) = 0.3846 -> kasin(theta) = 3.8317 -> a*scale_down = 0.8721(green), 1.11(red)

double eps_lens(const meep::vec &pos){ // radius of curvature 5um
    if (pos.y()<=(lens_ypos))
        if ((pos.x()-x_center)*(pos.x()-x_center) + (pos.y()-y_center)*(pos.y()-y_center) <= r_cur*r_cur) 
            return 1.5*1.5; // find the reference of the SiO2 refractive index
        else
            return 1.0;
    // else if ((pos.y()<=(lens_ypos + 1.0)) && (pos.y()>(lens_ypos)))
    //   return 1.5*1.5;
    else
      return 1.0;
}

double eps_free(const meep::vec &) { return 1.0; }

double** power_normalization(){
  double **power_norm = new double*[Nf_dft];
  grid_volume v_free = vol2d(Lx, Ly, resolution); // define volume
  structure *s_free = new structure(v_free, eps_free, pml(Lpml)); // define structure
  fields *f_free = new fields(s_free); //1st fields (free space)

  //1st plane for setting plane wave source
  Plane p = Plane(XDIM, 0, Lx, src_ypos, resolution); // define Plane for plane-wave
  const double k_nonp_f0 = sqrt(4. * PI * PI * f0_src * f0_src - k_per * k_per); // define magnitude of the k-vector
  Vec3 <std::complex<double> > k_inc(k_per, k_nonp_f0, 0); // define k-vector of the incidence wave
  // f_free->use_bloch(X, k_inc.x()/2./PI); //make sure you skip 2pi here****
  if (pol==0) // if TE polarization
    add_plane_wave(f_free, p, Ez, k_inc, f0_src, df_src);
  else { // else TM polarization
    add_plane_wave(f_free, p, Hz, k_inc, f0_src, df_src, cos(theta_f0));
      if (theta_f0>0)
        add_plane_wave(f_free, p, Ey, k_inc, f0_src, df_src, sin(theta_f0));
  }

  ////////////////////////////////////////////////////////////////////////////
  // free-space RGB flux for normalization
  vec l_bot(0, lens_ypos - 1.0); // define start corner for the position of the dft flux calculation
  vec r_bot(Lx, lens_ypos - 1.0); // define end corner for the position of the dft flux calculation
  // vec l_bot(Lx/2 - d, lens_ypos); // define start corner for the position of the dft flux calculation
  // vec r_bot(Lx/2 + d, lens_ypos); // define end corner for the position of the dft flux calculation
  volume vol_bot(l_bot, r_bot); // define the volume of the dft flux calculation 
  dft_flux flux_bot = f_free->add_dft_flux_plane(vol_bot, fmin_dft, fmax_dft, Nf_dft); // define the flux calculation setting such as target frequency, calculation position
  ////////////////////////////////////////////////////////////////////////////
  // wave propagation
  while (f_free->time() < (f_free->last_source_time() + Ly))
    f_free->step();
  ////////////////////////////////////////////////////////////////////////////
  // calculate free space flux and than write them on to TXT file
  double *power_bot = flux_bot.flux(); // save the flux data at power_bot
  power_norm[0] = new double[Nf_dft]; // make array
  for (int i=0; i<Nf_dft; ++i) // save the flux data at power_norm array
    power_norm[0][i] = abs(0.5 * power_bot[i]);

  meep::master_printf("flux_free=%f\n", *power_bot); // print the value of the free-space Red flux

  FILE *fp6;
  char name6[100];
  sprintf(name6,"output/flux/flux_free.txt"); // save the RGB flux txt file
  fp6 = fopen(name6,"w"); // This information is useful to understand how penalization affects to the results
  for(int i=0; i<Nf_dft; ++i){
    fprintf(fp6, "%e\n", power_norm[0][i]);
  }
  fclose(fp6);
////////////////////////////////////////////////////////////////////////////
  delete[] power_bot;
  return power_norm;
}

int main(int argc, char **argv) { // main optimization function
  meep::initialize mpi(argc, argv); // do this even for non-MPI Meep
  // double **power_norm = power_normalization();
  double** power_mat = new double*[1]; // make array for R monitor
  grid_volume v = vol2d(Lx, Ly, resolution); // define volume
  structure s(v, eps_lens, pml(Lpml)); // define structure
  fields f = fields(&s); // 1st field
  
  f.output_hdf5(Dielectric, v.surroundings());

  ////////////////////////////////////////////////////////////////////////////
  //1st plane for set the source
  Plane p = Plane(XDIM, 0, Lx, src_ypos, resolution);
  const double k_nonp_f0 = sqrt(4. * PI * PI * f0_src * f0_src - k_per * k_per);
  Vec3 <std::complex<double> > k_inc(k_per, k_nonp_f0, 0);
  // f.use_bloch(X, k_inc.x()/2./PI); //make sure you skip 2pi here****
  if (pol==0)
      add_plane_wave(&f, p, Ez, k_inc, f0_src, df_src);
  else {
      add_plane_wave(&f, p, Hz, k_inc, f0_src, df_src, cos(theta_f0));
      if (theta_f0>0)
          add_plane_wave(&f, p, Ey, k_inc, f0_src, df_src, sin(theta_f0));
  }
  ////////////////////////////////////////////////////////////////////////////

  ////////////////////////////////////////////////////////////////////////////

  vec l_bot0(Lx/2 + airy_NA*2.0, lens_ypos - focal); // 
  vec r_bot0(Lx/2 - airy_NA*2.0, lens_ypos - focal);
  volume vol_bot0(l_bot0, r_bot0); // volume at the first zero airy disk
  dft_flux flux_bot0 = f.add_dft_flux_plane(vol_bot0, fmin_dft, fmax_dft, Nf_dft);

  vec l_bot1(Lx/2 + airy_NA, lens_ypos - focal); // 
  vec r_bot1(Lx/2 - airy_NA, lens_ypos - focal); // 
  // vec l_bot1(Lx/2 + airy_NA*2.0, 35.85); // 
  // vec r_bot1(Lx/2 - airy_NA*2.0, 35.85); // 
  volume vol_bot1(l_bot1, r_bot1); // volume at the first zero airy disk
  dft_flux flux_bot1 = f.add_dft_flux_plane(vol_bot1, fmin_dft, fmax_dft, Nf_dft);

  vec l_bot2(0.0, lens_ypos - focal); // output flux
  vec r_bot2(Lx, lens_ypos - focal);
  volume vol_bot2(l_bot2, r_bot2); // volume at the first zero airy disk
  dft_flux flux_bot2 = f.add_dft_flux_plane(vol_bot2, fmin_dft, fmax_dft, Nf_dft);

  vec l_bot3(0.0, lens_ypos - (r_cur - sqrt(r_cur * r_cur - d * d)) - 1.0); // transmitted flux 
  vec r_bot3(Lx, lens_ypos - (r_cur - sqrt(r_cur * r_cur - d * d)) - 1.0); //  
  volume vol_bot3(l_bot3, r_bot3); // volume at the first zero airy disk
  dft_flux flux_bot3 = f.add_dft_flux_plane(vol_bot3, fmin_dft, fmax_dft, Nf_dft);

// monitor for PML absorption
  vec l_bot_x1(0.1, 0.0); 
  vec r_bot_x1(0.1, Ly);
  volume vol_bot_x1(l_bot_x1, r_bot_x1); // volume at the first zero airy disk
  dft_flux flux_bot_x1 = f.add_dft_flux_plane(vol_bot_x1, fmin_dft, fmax_dft, Nf_dft);

  vec l_bot_x2(Lx - 0.1, 0.0); 
  vec r_bot_x2(Lx - 0.1, Ly);
  volume vol_bot_x2(l_bot_x2, r_bot_x2); // volume at the first zero airy disk
  dft_flux flux_bot_x2 = f.add_dft_flux_plane(vol_bot_x2, fmin_dft, fmax_dft, Nf_dft);
  
  // vec l_bot_x3(0.0, Ly - 3.0); 
  // vec r_bot_x3(Lx, Ly - 3.0);
  // volume vol_bot_x3(l_bot_x3, r_bot_x3); // volume at the first zero airy disk
  // dft_flux flux_bot_x3 = f.add_dft_flux_plane(vol_bot_x3, fmin_dft, fmax_dft, Nf_dft);

  // vec l_bot_x4(0.0, src_ypos - 1.0); 
  // vec r_bot_x4(Lx, src_ypos - 1.0);
  // volume vol_bot_x4(l_bot_x4, r_bot_x4); // volume at the first zero airy disk
  // dft_flux flux_bot_x4 = f.add_dft_flux_plane(vol_bot_x4, fmin_dft, fmax_dft, Nf_dft);


// define each component of the electric and magnetic fields
  meep::component components[6] = {Ex, Ey, Ez, Hx, Hy, Hz};
// 렌즈를 뺀 아랫부분 field, flux monitor
  vec l_bot5(Lpml, 0);
  vec r_bot5(Lx-Lpml, lens_ypos - (r_cur - sqrt(r_cur * r_cur - d * d)));
  volume vol_bot5(l_bot5, r_bot5);
  dft_fields fields_bot = f.add_dft_fields(components, 6, vol_bot5, fmin_dft, fmax_dft, Nf_dft);
  dft_flux flux_bot4 = f.add_dft_flux(Y, vol_bot5, fmin_dft, fmax_dft, Nf_dft); // flux_bot4.flux는 필요없나?
//////////////////////////////wave propagation//////////////////////////////
   while (f.time() < (f.last_source_time() + Ly * 1.25)){
      f.step();
      // if (pol == 1){
      //   if (std::fmod(f.time(), 0.25) == 0.0)
      //     f.output_hdf5(meep::Ex, vol_bot5, 0, false, true, "output/snapshot/Ex");
      // }
      // else {
      //   if (std::fmod(f.time(), 0.25) == 0.0)
      //     f.output_hdf5(meep::Ez, vol_bot5, 0, false, true, "output/snapshot/Ez");
      // }
    }
  ////////////////////////////////////////////////////////////////////////////
  //calculate Flux + write them on TXT_file
  double *power_bot0 = flux_bot0.flux();
  double *power_bot = flux_bot1.flux();
  double *power_bot2 = flux_bot2.flux();
  double *power_bot3 = flux_bot3.flux();
  double *power_pml1 = flux_bot_x1.flux();
  double *power_pml2 = flux_bot_x2.flux();
  // double *power_ref = flux_bot_x3.flux();
  // double *power_input = flux_bot_x4.flux();

  power_mat[0] = new double[Nf_dft];
  for (int i=0; i<Nf_dft; ++i)
    power_mat[0][i] = abs(0.5 * power_bot0[i]);

    meep::master_printf("NA0.4_airy_disk_flux=%f\n", *power_bot0); // 
    meep::master_printf("Focal_point_flux=%f\n", *power_bot); // 
    meep::master_printf("output_flux=%f\n", *power_bot2); // 
    meep::master_printf("transmitted_flux=%f\n", *power_bot3); // 
    meep::master_printf("Left_PML_flux=%f\n", *power_pml1); // 
    meep::master_printf("Right_PML_flux=%f\n", *power_pml2); // 
    // meep::master_printf("Reflection_flux=%f\n", *power_ref); // 
    // meep::master_printf("input_flux=%f\n", *power_input); // 



// save the flux data txt file
    FILE *fp6;
    char name6[100];
    sprintf(name6,"output/flux/flux_airy_disk.txt"); // save the Red flux data txt file at airy disk
    fp6 = fopen(name6,"w"); // This information is useful to understand how penalization affects to the results
    for(int i=0; i<Nf_dft; ++i){
      fprintf(fp6, "%e\n", power_mat[0][i]);
    }
    fclose(fp6);

  if (pol == 0){ // save the field data file
    f.output_dft(flux_bot4, "output/h5/dft-flux");
    f.output_dft(fields_bot, "output/h5/dft-fields");
  }
  else{
    f.output_dft(flux_bot4, "output/h5/dft-flux");
    f.output_dft(fields_bot, "output/h5/dft-fields");
  }

  delete[] power_mat[0];
  delete[] power_mat;

  return 0;
}